In [1]:
import sys
import os

# Add the path to your ML directory
sys.path.append(r'C:\Users\admin\Documents\Masters\ES_Masters\Masters-Processing\ML')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import optuna.visualization as vis
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.feature_selection import RFE
from mrmr_wrapper import MRMRTransformer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import optuna
import optuna.visualization as vis
from sklearn.pipeline import Pipeline

# Import Data

In [2]:
# Load the data
file_path = "../features-Master.csv"
data = pd.read_csv(file_path)

# Shuffle the data
shuffled = data.sample(frac=1, random_state=42).reset_index(drop=True)
data_shuffled = shuffled.iloc[:, 4:]
labels_shuffled = shuffled["Comfort Score"]

# Train/Test Split

In [3]:
# For regression (using stratified split based on binned target)
X_train, X_test, y_train, y_test = train_test_split(
    data_shuffled, 
    labels_shuffled, 
    test_size=0.2, 
    stratify=labels_shuffled,
    random_state=42
)

# Optimize Feature Selection for and Parameters for RF Model

In [4]:
X = X_train.copy()
y = y_train.copy()

def objective(trial):
    # Feature selection, only optimizing method and number of features (no hyperparameters of the methods)
    fs_method = trial.suggest_categorical('feature_selection', ['RFE', 'MRMR', 'None'])

    if fs_method != 'None':
        k_features = trial.suggest_int('k_features', 5, 105, step = 10) 
        if fs_method == 'RFE':
            estimator = RandomForestRegressor()
            selector = RFE(estimator, n_features_to_select=k_features)
        else: #MRMR
            selector = MRMRTransformer(k_features=k_features) #https://feature-engine.trainindata.com/en/1.8.x/api_doc/selection/MRMR.html#feature_engine.selection.MRMR
    else:
        selector = 'passthrough'

    # Random Forest Hyperparameters
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
        'random_state': 42
    }
    model = RandomForestRegressor(**params)

    # 3. Pipeline 
    pipeline = Pipeline([
        ('feature_selection', selector),
        ('model', model)
    ])

    #4. Cross-validation
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    try:
        scores = cross_val_score(pipeline, X, y, cv=cv, scoring='r2', n_jobs=1)
        return np.mean(scores)
    except Exception:
        return -np.inf

# Run Optuna Study
study = optuna.create_study(direction='maximize') 
study.optimize(objective, n_trials=5, show_progress_bar=True, n_jobs=6) 

# Best result
print("\n Regression Optimization Results")
print("===================================")
print(f"Best R²: {study.best_value:.4f}")
print("Best Parameters: ")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

[I 2025-09-16 14:31:47,188] A new study created in memory with name: no-name-13874db4-0a3c-4aa5-910d-32fb1d4af83e


  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:01<00:00, 23.81it/s]


Got MRMR features


100%|██████████| 25/25 [00:00<00:00, 25.96it/s]


...

Got MRMR features


100%|██████████| 25/25 [00:00<00:00, 25.99it/s]


Got MRMR features


100%|██████████| 25/25 [00:01<00:00, 24.25it/s]


Got MRMR features
[I 2025-09-16 14:32:11,103] Trial 4 finished with value: 0.165051466613897 and parameters: {'feature_selection': 'MRMR', 'k_features': 25, 'n_estimators': 815, 'max_depth': 19, 'min_samples_split': 7, 'min_samples_leaf': 4}. Best is trial 4 with value: 0.165051466613897.
[I 2025-09-16 14:32:17,979] Trial 2 finished with value: 0.1948677822404677 and parameters: {'feature_selection': 'None', 'n_estimators': 222, 'max_depth': 9, 'min_samples_split': 7, 'min_samples_leaf': 4}. Best is trial 2 with value: 0.1948677822404677.
[I 2025-09-16 14:33:09,540] Trial 0 finished with value: 0.200251337588446 and parameters: {'feature_selection': 'None', 'n_estimators': 607, 'max_depth': 18, 'min_samples_split': 10, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.200251337588446.
[I 2025-09-16 15:09:34,188] Trial 3 finished with value: 0.2032048823924865 and parameters: {'feature_selection': 'RFE', 'k_features': 75, 'n_estimators': 396, 'max_depth': 10, 'min_samples_split': 8,

# Visualization

In [5]:
# 1. Parameter Importance Plot
fig = vis.plot_param_importances(study, target_name="R2 Score")
fig.show()

# 2. Slice Plot (Best for seeing individual parameter effects)
fig = vis.plot_slice(
    study,
    params=['max_depth','n_estimators','k_features'],
    target_name="R2 Score"
)
fig.show()

# Apply Feature Selection to Training & Test Data

In [6]:
# apply your feature selection code from before
best_fs_method = study.best_params.get('feature_selection', 'None')

if best_fs_method != 'None':
    k_features = study.best_params['k_features']
    if best_fs_method == 'RFE':
        estimator = RandomForestRegressor()
        selector = RFE(estimator, n_features_to_select=k_features,)
    else: #MRMR
        selector = MRMRTransformer(k_features=k_features)
    
    selector.fit(X, y)
    if hasattr(selector, 'get_support'):  # For SelectKBest/RFE
        selected_features = X.columns[selector.get_support()]
    else:  # For MRMR
        selected_features = selector.selected_features
    X_best = X[selected_features]
else:
    X_best = X
    selected_features = X.columns

# Apply the same feature selection to test data
if best_fs_method != 'None':
    if best_fs_method == 'MRMR':
        X_test_final = X_test[selected_features]
    else:
        X_test_final = selector.transform(X_test)  # Use the already fitted selector
        if isinstance(X_test, pd.DataFrame):
            X_test_final = pd.DataFrame(X_test_final, columns=selected_features)
else:
    X_test_final = X_test

# Create and Fit RF Model with Best Parameters

In [7]:
best_model = RandomForestRegressor(
    n_estimators=study.best_params['n_estimators'],
    max_depth=study.best_params['max_depth'],
    min_samples_split=study.best_params['min_samples_split'],
    min_samples_leaf=study.best_params['min_samples_leaf'],
    random_state=42
)

best_model.fit(X_best, y)

RandomForestRegressor(max_depth=10, min_samples_leaf=4, min_samples_split=8,
                      n_estimators=396, random_state=42)

# Run Model and Get Outcomes

In [8]:
# Make predictions
y_pred = best_model.predict(X_test_final)

# Calculate metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\nFinal Model Evaluation on Test Set:")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")

# Calculate accuracy within ±1 point
correct = np.sum(np.abs(y_test - y_pred) <= 1)
accuracy = correct / len(y_test)
print(f"Accuracy within ±1 point: {accuracy:.4f}")


Final Model Evaluation on Test Set:
RMSE: 1.1708
R²: 0.1459
Accuracy within ±1 point: 0.5500
